In [25]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

import nltk
import torch

In [38]:
horos = pd.read_csv('horoscopes_SA.csv', 
                         sep=';',
                         on_bad_lines='skip',
                         names=["ID", "ROBERTA", "DATE", "SIGN", "VADERS"],
                         skiprows=1)
horos['DATE'] = pd.to_datetime(horos.DATE).dt.strftime('%Y-%m-%d')
print(horos.head(3))

   ID   ROBERTA        DATE   SIGN  VADERS
0   0 -0.224260  2013-12-01  aries  0.7102
1   1 -0.250313  2013-12-02  aries  0.3400
2   2  0.716438  2013-12-03  aries  0.4939


In [39]:
stocks = pd.read_csv('historical_stock_prices.csv', 
                         sep=',',
                         on_bad_lines='skip',
                         names=["TICKER", "OPEN", "CLOSE", "ADJ_CLOSE", "LOW", "HIGH", "VOLUME", "DATE"],
                         skiprows=1)
print(stocks.head(3))

  TICKER   OPEN  CLOSE  ADJ_CLOSE    LOW   HIGH   VOLUME        DATE
0    AHH  11.50  11.58   8.493155  11.25  11.68  4633900  2013-05-08
1    AHH  11.66  11.55   8.471151  11.50  11.66   275800  2013-05-09
2    AHH  11.55  11.60   8.507822  11.50  11.60   277100  2013-05-10


In [40]:
tickers = stocks.groupby("TICKER").DATE.agg(["min"]).rename(columns={"min": "BD"}) 
tickers.head(3)

,BD
TICKER,
A,1999-11-18
AA,1970-01-02
AABA,1996-04-12


In [41]:
minhoros = min(horos['DATE'])
maxhoros = max(horos['DATE'])
stocks = stocks[(stocks['DATE'] >= minhoros) & (stocks['DATE'] <= maxhoros)]

In [42]:
import random

def zodiac_sign(day, month):
    signs = ["capricorn", "aquarius", "pisces", "aries", "taurus", "gemini", "cancer", 
             "leo", "virgo", "libra", "scorpio", "sagittarius"]
    
    # Choose a random zodiac sign
    chosen_sign = random.choice(signs)
    
    # Get a random date for that sign
    if chosen_sign == "capricorn":
        day = random.randint(22, 31) if random.choice([True, False]) else random.randint(1, 19)
        month = 12 if day > 21 else 1
    elif chosen_sign == "aquarius":
        day = random.randint(20, 31) if random.choice([True, False]) else random.randint(1, 18)
        month = 1 if day > 19 else 2
    # ... (repeat for all signs)
    elif chosen_sign == "sagittarius":
        day = random.randint(22, 30) if random.choice([True, False]) else random.randint(1, 21)
        month = 11 if day > 21 else 12
    
    return chosen_sign

In [43]:
tickers['BD'] = pd.to_datetime(tickers['BD'])
tickers['SIGN'] = tickers['BD'].apply(lambda x: zodiac_sign(x.day, x.month))
tickers.head(3)

,BD,SIGN
TICKER,,
A,1999-11-18,cancer
AA,1970-01-02,pisces
AABA,1996-04-12,aquarius


In [44]:
stocks = stocks.merge(tickers[['SIGN']], on='TICKER', how='left')
stocks.head(3)

,TICKER,OPEN,CLOSE,ADJ_CLOSE,LOW,HIGH,VOLUME,DATE,SIGN
0,AHH,10.21,9.88,7.409460,9.84,10.22,53000,2013-12-02,virgo
1,AHH,9.84,10.09,7.566949,9.66,10.21,102900,2013-12-03,virgo
2,AHH,10.08,10.04,7.529452,9.90,10.28,80900,2013-12-04,virgo


In [45]:
merged = stocks.merge(horos[["DATE", "SIGN", "ROBERTA", "VADERS"]], on=['DATE', 'SIGN'], how='left')
merged.head(3)
merged = merged.drop(columns=['OPEN', 'CLOSE', 'LOW'])

In [46]:
merged = merged.sort_values(by=['TICKER', 'DATE'])
merged['ADJ_CLOSE'] = pd.to_numeric(merged['ADJ_CLOSE'], errors='coerce')
merged['HIGH'] = pd.to_numeric(merged['HIGH'], errors='coerce')
merged['VOLUME'] = pd.to_numeric(merged['VOLUME'], errors='coerce')

merged['ADJ_CLOSE_diff'] = merged.groupby('TICKER')['ADJ_CLOSE'].diff()
merged['HIGH_diff'] = merged.groupby('TICKER')['HIGH'].diff()
merged['VOLUME_diff'] = merged.groupby('TICKER')['VOLUME'].diff()
merged['ADJ_CLOSE_NORM'] = merged.groupby('TICKER').apply(lambda x: (x['ADJ_CLOSE_diff'] / x['ADJ_CLOSE'].shift(1)) * 100).reset_index(level=0, drop=True)
merged['HIGH_NORM'] = merged.groupby('TICKER').apply(lambda x: (x['HIGH_diff'] / x['HIGH'].shift(1)) * 100).reset_index(level=0, drop=True)
merged['VOLUME_NORM'] = merged.groupby('TICKER').apply(lambda x: (x['VOLUME_diff'] / x['VOLUME'].shift(1)) * 100).reset_index(level=0, drop=True)

merged = merged.drop(columns=['ADJ_CLOSE', 'HIGH', 'VOLUME','ADJ_CLOSE_diff', 'HIGH_diff', 'VOLUME_diff'])
merged = merged.rename(columns={"ADJ_CLOSE_NORM": "ADJ_CLOSE", "VOLUME_NORM": "VOLUME", "HIGH_NORM": "HIGH"})
merged

,TICKER,DATE,SIGN,ROBERTA,VADERS,ADJ_CLOSE,HIGH,VOLUME
2304675,A,2013-12-02,cancer,0.305432,0.9286,NaN,NaN,NaN
2304676,A,2013-12-03,cancer,-0.273571,-0.5106,-0.526240,-0.931611,69.748517
2304680,A,2013-12-04,cancer,0.533749,0.8595,1.228044,1.241302,-2.469171
2304681,A,2013-12-05,cancer,-0.052018,-0.4215,-0.149309,-0.055737,-25.059221
2304685,A,2013-12-06,cancer,-0.127950,-0.2960,2.485975,2.100378,68.655419
...,...,...,...,...,...,...,...,...
1834702,ZYNE,2016-12-06,cancer,-0.359017,0.7543,9.711682,4.571431,49.317073
1834705,ZYNE,2016-12-07,cancer,-0.560320,-0.5267,0.345783,0.136609,-39.268213
1834708,ZYNE,2016-12-08,cancer,0.252714,0.4215,7.029631,7.230562,74.771382
1834711,ZYNE,2016-12-09,cancer,-0.127943,-0.4118,-2.962009,5.534350,7.510003


In [49]:
def normalize(series):
    mins = series.min()
    maxs = series.max()
    return 2 * (series - mins) / (maxs - mins) - 1

In [16]:
merged['ADJ_CLOSE_NORM'] = merged.groupby('TICKER')['ADJ_CLOSE'].transform(normalize)
merged['HIGH_NORM'] = merged.groupby('TICKER')['HIGH'].transform(normalize)
merged['VOLUME_NORM'] = merged.groupby('TICKER')['VOLUME'].transform(normalize)
merged

,TICKER,DATE,SIGN,ROBERTA,VADERS,ADJ_CLOSE,HIGH,VOLUME
2304675,A,2013-12-02,sagittarius,0.270197,0.8883,NaN,NaN,NaN
2304676,A,2013-12-03,sagittarius,0.889425,0.5989,0.221234,-0.353376,-0.475777
2304680,A,2013-12-04,sagittarius,-0.738974,-0.7184,0.506390,0.060971,-0.737473
2304681,A,2013-12-05,sagittarius,0.843855,0.7755,0.282503,-0.186358,-0.819333
2304685,A,2013-12-06,sagittarius,0.172897,-0.5321,0.710865,0.224786,-0.479738
...,...,...,...,...,...,...,...,...
1834702,ZYNE,2016-12-06,gemini,-0.715441,0.5378,-0.543567,-0.587971,-0.897368
1834705,ZYNE,2016-12-07,gemini,-0.631642,0.9022,-0.647231,-0.635200,-0.963182
1834708,ZYNE,2016-12-08,gemini,0.662621,0.5090,-0.573252,-0.559652,-0.878457
1834711,ZYNE,2016-12-09,gemini,0.990157,0.9665,-0.683842,-0.577716,-0.928428


In [47]:
merged['ROBERTA'] = merged['ROBERTA'].astype(float)
merged['VADERS'] = merged['VADERS'].astype(float)
merged['DATE'] = pd.to_datetime(merged['DATE']).dt.strftime('%Y-%m-%d')
merged['TICKER'] = merged['TICKER'].astype("string")
merged['SIGN'] = merged['SIGN'].astype("string")
merged.dtypes

TICKER       string[python]
DATE                 object
SIGN         string[python]
ROBERTA             float64
VADERS              float64
ADJ_CLOSE           float64
HIGH                float64
VOLUME              float64
dtype: object

In [48]:
merged.to_csv('db_false.csv', sep=';')